In [1]:
#神经网络可以使用torch.nn来构建神经网络。
#现在，我们已经对autograd有了一些了解，神经网络是基于自动梯度（autograd）来定义一些模型。一个nn.Module包括层和一个方法forward（input），它会返回输出（output）。
#下面我们来看一个对数字图像进行分类的网络：MNIST数据集，卷积网络
#卷积网络是一个简单的前馈网络，它获取输入，将其一层又一层地馈入，然后最终输出。
'''
神经网络的典型训练过程如下：
1.定义具有一些可学习参数（或权重）的神经网络
2.遍历输入数据集
3.通过网络处理输入
4.计算损失（输出与正确的距离有多远）
5.将梯度传播回网络参数
6.通常使用简单的更新规则来更新网络的权重：weight=weight-learning_rate*gradient
'''

'\n神经网络的典型训练过程如下：\n1.定义具有一些可学习参数（或权重）的神经网络\n2.遍历输入数据集\n3.通过网络处理输入\n4.计算损失（输出与正确的距离有多远）\n5.将梯度传播回网络参数\n6.通常使用简单的更新规则来更新网络的权重：weight=weight-learning_rate*gradient\n'

In [2]:
#定义网络
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel,6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
# 现在只需要定义forward函数，backward就可以使用自动定义函数（计算梯度）autograd。
# 你可以在forward函数中使用任何Tensor操作
# 一个模型可训练的参数可以通过调用net.parameters()返回
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's.weight

10
torch.Size([6, 1, 3, 3])


In [6]:
# 我们尝试随机生成一个32x32的输入。注意期望的输入维度是32x32.为了使这个网络在MNIST数据集上，需要把数据集中的图片维度修改为32x32.
input = torch.randn(1, 1, 32, 32)

In [7]:
input

tensor([[[[-3.0343,  1.7212,  0.8719,  ...,  0.4716,  0.3469, -1.5875],
          [ 1.1123, -1.9776,  1.3877,  ...,  0.9826,  0.7551, -0.5930],
          [-0.3505, -1.2469, -0.4147,  ...,  0.9005, -0.4487,  1.3531],
          ...,
          [ 0.0094, -0.4040,  1.8994,  ...,  0.5343,  1.4669,  0.9121],
          [-1.6977,  0.3063, -0.7818,  ..., -1.3234, -0.8817,  0.1485],
          [ 0.1468, -0.8137, -0.4283,  ..., -1.0772,  0.2471,  1.1040]]]])

In [8]:
out = net(input)
print(out)

tensor([[ 0.0556, -0.0037,  0.0612, -0.1138, -0.0826, -0.0122, -0.1700,  0.0042,
         -0.0230,  0.1461]], grad_fn=<AddmmBackward>)


In [9]:
# 把所有参数梯度缓冲器置0，用随机的梯度来反向传播
net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
out

tensor([[ 0.0556, -0.0037,  0.0612, -0.1138, -0.0826, -0.0122, -0.1700,  0.0042,
         -0.0230,  0.1461]], grad_fn=<AddmmBackward>)

In [12]:
input.grad

In [13]:
input

tensor([[[[-3.0343,  1.7212,  0.8719,  ...,  0.4716,  0.3469, -1.5875],
          [ 1.1123, -1.9776,  1.3877,  ...,  0.9826,  0.7551, -0.5930],
          [-0.3505, -1.2469, -0.4147,  ...,  0.9005, -0.4487,  1.3531],
          ...,
          [ 0.0094, -0.4040,  1.8994,  ...,  0.5343,  1.4669,  0.9121],
          [-1.6977,  0.3063, -0.7818,  ..., -1.3234, -0.8817,  0.1485],
          [ 0.1468, -0.8137, -0.4283,  ..., -1.0772,  0.2471,  1.1040]]]])

In [14]:
#在此，我们完成了：
#1.定义一个神经网络
#2.处理输入以及调用反向传播
#还剩下：
#1.计算损失值
#2.更新网络中的权重
'''
损失函数
一个损失函数需要一对输入：模型输出和目标，然后计算一个值来评估输出距离目标有多远。
有一些不同的损失函数在nn包中。一个简单的损失函数就是nn.MSELoss,这计算了均方误差。
'''
# 例如：
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9975, grad_fn=<MseLossBackward>)


In [15]:
#现在，如果跟随损失到反向传播路径，可以使用它的.grad_fn属性，你将会看到一个这样的计算图：
'''
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
'''
#所以，当我们调用loss.backward(),整个图都会微分，而且所有的在途中的requires_grad=True
#的张量将会让他们的grad张量累计梯度。
#为了掩饰，我们将跟随以下步骤来反向传播
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [16]:
'''
反向传播
为了实现反向传播损失，我们所有需要做的事情仅仅是使用loss.backward().
我们需要清空现存的梯度，要不然都将会和现存的梯度累计到一起。
现在我们调用loss.backward(),然后看一下conv1的偏置项在反向传播之前和之后的变化。
'''

net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv2.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0144, -0.0006, -0.0012,  0.0148, -0.0080,  0.0107,  0.0076,  0.0208,
         0.0179, -0.0023, -0.0218,  0.0062,  0.0077, -0.0032, -0.0020, -0.0017])


In [25]:
'''
现在我们知道了如何使用损失函数进行反向传播，现在唯一剩下的事情就是更新
神经网络的参数。
更新神经网络参数：最简单的更新规则就是随机梯度下降。
weight = weight - learning_rate * gradient
我们可以使用python来实现这个规则
'''
learning_rate = 0.01
for f in net.parameters():
    print(f.data)
    f.data.sub_(f.grad.data * learning_rate)
    print(f.data)
    

tensor([[[[-0.0502, -0.2061, -0.1162],
          [ 0.2882,  0.1802,  0.2523],
          [ 0.2002, -0.1097,  0.1336]]],


        [[[-0.2733, -0.2333, -0.1124],
          [ 0.0679,  0.0111,  0.1859],
          [ 0.2323,  0.2205,  0.2763]]],


        [[[-0.1103, -0.1537, -0.0846],
          [-0.1935,  0.1479,  0.0989],
          [ 0.2904,  0.0532, -0.1410]]],


        [[[ 0.3094, -0.1010,  0.3323],
          [ 0.2727, -0.1611,  0.3113],
          [-0.1852,  0.1348,  0.0894]]],


        [[[-0.0931,  0.1538, -0.1028],
          [ 0.3048,  0.2448, -0.3126],
          [ 0.0726,  0.1932, -0.1772]]],


        [[[-0.1150,  0.2831,  0.1790],
          [ 0.1379,  0.0381, -0.1793],
          [-0.3255,  0.2705,  0.1319]]]])
tensor([[[[-0.0503, -0.2060, -0.1161],
          [ 0.2880,  0.1803,  0.2523],
          [ 0.2003, -0.1096,  0.1335]]],


        [[[-0.2731, -0.2332, -0.1124],
          [ 0.0679,  0.0112,  0.1858],
          [ 0.2324,  0.2203,  0.2761]]],


        [[[-0.1102, -0.1538, -0.0

In [23]:
params

[Parameter containing:
 tensor([[[[-0.0501, -0.2062, -0.1162],
           [ 0.2883,  0.1801,  0.2524],
           [ 0.2001, -0.1098,  0.1337]]],
 
 
         [[[-0.2736, -0.2333, -0.1123],
           [ 0.0679,  0.0111,  0.1860],
           [ 0.2321,  0.2206,  0.2765]]],
 
 
         [[[-0.1105, -0.1535, -0.0847],
           [-0.1936,  0.1479,  0.0990],
           [ 0.2903,  0.0530, -0.1410]]],
 
 
         [[[ 0.3094, -0.1011,  0.3321],
           [ 0.2726, -0.1609,  0.3112],
           [-0.1853,  0.1349,  0.0893]]],
 
 
         [[[-0.0930,  0.1538, -0.1028],
           [ 0.3048,  0.2449, -0.3126],
           [ 0.0723,  0.1931, -0.1771]]],
 
 
         [[[-0.1151,  0.2831,  0.1790],
           [ 0.1379,  0.0382, -0.1796],
           [-0.3255,  0.2706,  0.1321]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.0502,  0.2766, -0.2860,  0.0122, -0.2435, -0.0188],
        requires_grad=True),
 Parameter containing:
 tensor([[[[ 6.6788e-02, -9.5214e-02, -5.8560e-02],
           [

In [26]:
'''
尽管如此，如果你使用神经网络，想使用不同的更新规则，类似于SGD、Nesterov-SGD、Adam、RMSProp等。为了让这可行,
我们建立了一个小包：torch.optim实现了所有的方法，使用它非常的简单。
'''
import torch.optim as optim

In [27]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()